In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import time
import json
import pandas as pd
import numpy as np
import os
import datetime
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods

In [32]:
# file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/sudden_drift/BPIC17_new.csv'
# file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/test_dataset/DomesticDeclarations_mini.csv'
# file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/sudden_drift/Hospital+BPI12_new.csv'
file='/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/sudden_drift/BPIC15ALL.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

# Extract the filename from the file path
filename = os.path.basename(file)

# Remove the file extension from the filename
dataName = os.path.splitext(filename)[0]

#dataName = 'Helpdesk_mini'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
print('data prepared done')

Num events is 262628


/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/ipykernel_launcher.py:18: DtypeWarning: Columns (6,22) have mixed types.Specify dtype option on import or set low_memory=False.
/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/ipykernel_launcher.py:21: DtypeWarning: Columns (6,22) have mixed types.Specify dtype option on import or set low_memory=False.


PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 0
Test: 262628
data prepared done


### make mini dataset

In [6]:
data_mini = data.iloc[:300, :]
data_mini.to_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/test_dataset/BPIC15ALL_mini.csv', index=False)

In [17]:
connect_symbol="-"
if '/' in d.logfile.get_data()['completeTime'][0]:
    connect_symbol='/'
formats = [f"%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S%z", f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S',f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S.%f']

for timeformat in formats:
    try:
        pd.to_datetime(d.logfile.get_data()['completeTime'], format=timeformat,exact=True)
        print(d.logfile.get_data()['completeTime'])
        print("The time format is:", timeformat)
        break
    except ValueError:
        continue
print('timeformat',timeformat)

0         2009-11-18 00:00:00
1         2009-11-23 00:00:00
2         2010-01-01 00:00:00
3         2010-05-13 00:00:00
4         2010-06-29 00:00:00
                 ...         
262623    2015-03-05 00:00:00
262624    2015-03-05 00:00:00
262625    2015-03-09 00:00:00
262626    2015-03-09 00:00:00
262627    2015-08-01 00:00:00
Name: completeTime, Length: 262628, dtype: object
The time format is: %Y-%m-%d %H:%M:%S
timeformat %Y-%m-%d %H:%M:%S


### check monthly batch size

In [33]:
connect_symbol="-"
if '/' in d.logfile.get_data()['completeTime'][0]:
    connect_symbol='/'
formats = [f"%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S%z", f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S',f'%Y{connect_symbol}%m{connect_symbol}%d %H:%M:%S.%f']

for timeformat in formats:
    try:
        pd.to_datetime(d.logfile.get_data()['completeTime'], format=timeformat,exact=True)
        print(d.logfile.get_data()['completeTime'])
        print("The time format is:", timeformat)
        break
    except ValueError:
        continue
print('timeformat',timeformat)

0         2009-11-18 00:00:00
1         2009-11-23 00:00:00
2         2010-01-01 00:00:00
3         2010-05-13 00:00:00
4         2010-06-29 00:00:00
                 ...         
262623    2015-03-05 00:00:00
262624    2015-03-05 00:00:00
262625    2015-03-09 00:00:00
262626    2015-03-09 00:00:00
262627    2015-08-01 00:00:00
Name: completeTime, Length: 262628, dtype: object
The time format is: %Y-%m-%d %H:%M:%S
timeformat %Y-%m-%d %H:%M:%S


In [34]:
d.create_batch(split='month',timeformat=timeformat)

In [35]:
month_batch_size_list = []  # the number of activities in each month
for key,value in d.test.items():
    print(key,value['data'].get_data().shape[0])
    month_batch_size_list.append(value['data'].get_data().shape[0])
month_batch_size = np.average(np.array(month_batch_size_list))
month_batch_size

2009/11 2
2010/01 1
2010/05 1
2010/06 1
2010/10 821
2010/11 2344
2010/12 4119
2011/01 4924
2011/02 4813
2011/03 5713
2011/04 5826
2011/05 6144
2011/06 6591
2011/07 5395
2011/08 5564
2011/09 4911
2011/10 5852
2011/11 5201
2011/12 5475
2012/01 5218
2012/02 4997
2012/03 5770
2012/04 4385
2012/05 4092
2012/06 5144
2012/07 4881
2012/08 4992
2012/09 4911
2012/10 5606
2012/11 5167
2012/12 4608
2013/01 5963
2013/02 4203
2013/03 7013
2013/04 5273
2013/05 5371
2013/06 6311
2013/07 3493
2013/08 5143
2013/09 4256
2013/10 5145
2013/11 4865
2013/12 4366
2014/01 4216
2014/02 4576
2014/03 4878
2014/04 4073
2014/05 4380
2014/06 4484
2014/07 5513
2014/08 3376
2014/09 5337
2014/10 5388
2014/11 4934
2014/12 4907
2015/01 5385
2015/02 5583
2015/03 726
2015/08 1


4451.322033898305

In [27]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/sudden_drift/Hospital+BPI12_new.csv'
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/sudden_drift/BPIC17_new.csv'

In [36]:
def average_monthly_events(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert the "completeTime" column to a datetime object
    df['completeTime'] = pd.to_datetime(df['completeTime'], errors='coerce')

    # Extracting the month and year from the "completeTime" column
    df['month_year'] = df['completeTime'].dt.to_period('M')

    # Grouping by the month and year and counting the number of events
    monthly_events = df.groupby('month_year').size()

    # Calculating the average number of events per month
    avg_events_per_month = monthly_events.mean()

    return avg_events_per_month

In [37]:
average_monthly_events(file)

/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (6,22) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


4451.322033898305

In [45]:
def get_drift_locations(file_path):
    # Extracting data name from the file path
    data_name = file_path.split('/')[-1].replace('.csv', '')

    # Reading the CSV file
    df = pd.read_csv(file_path)

    # Finding the unique cases and calculating the drift interval
    num_unique_cases = df['case'].nunique()
    print("Num unique cases:", num_unique_cases)
    drift_interval = num_unique_cases // 10

    # Finding the drift locations based on the drift interval
    drift_locations = []
    case_count = 0
    previous_case = None
    for index, row in df.iterrows():
        current_case = row['case']
        if current_case != previous_case:
            case_count += 1
            if case_count % drift_interval == 0:
                drift_locations.append(index)
        previous_case = current_case

    return data_name, drift_locations

### recursevely check dataset size and monthly batch size

In [47]:
import os
import csv

# Define the directory path
dir_path = "/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/synthetic_dataset"

# Iterate over all files in the directory
for filename in os.listdir(dir_path):
    # Check if the filename ends with "5000.csv"
    if filename.endswith("5000.csv"):
        # Construct the full path to the file
        full_path = os.path.join(dir_path, filename)
        # Count the number of rows without the header
        # with open(full_path, 'r') as file:
        #     reader = csv.reader(file)
        #     next(reader)  # Skip the header
        #     row_count = sum(1 for row in reader)
        #     print(f"{filename} has {row_count} rows (excluding header).")
        # ame = average_monthly_events(full_path)
        # print(f"{filename}: {ame}")
        dl = get_drift_locations(full_path)
        print(filename, dl)
        # Now you can process the file using its full path


Num unique cases: 5000
OIR5000.csv ('OIR5000', [5212, 11033, 16235, 22165, 27314, 33174, 38341, 44256, 49443, 55322])
Num unique cases: 5000
ROI5000.csv ('ROI5000', [5212, 10354, 15555, 20740, 25888, 31192, 36354, 41555, 46737, 51987])
Num unique cases: 5000
IOR5000.csv ('IOR5000', [5212, 10460, 15662, 20990, 26138, 31420, 36583, 41841, 47027, 52311])
Num unique cases: 5000
ORI5000.csv ('ORI5000', [5212, 10988, 16189, 21939, 27089, 32943, 38111, 43746, 48934, 54604])
Num unique cases: 5000
RIO5000.csv ('RIO5000', [5212, 10663, 15868, 21403, 26553, 32105, 37273, 42765, 47953, 53418])
Num unique cases: 5000
IRO5000.csv ('IRO5000', [5212, 10741, 15949, 21449, 26593, 32154, 37316, 42941, 48123, 53639])
